In [1]:
import os
import sys
import math


import fiona

import numpy as np
import pandas as pd
import geopandas as gpd
from geopandas.tools import sjoin

from shapely.wkt import dumps, loads
from shapely.geometry import Point, box

## LFI

Getting data for the National Forest Inventory (Landes Forst Inventar LFI). There are separate informations about plot centers and individual trees per plot. The information about individual trees is split into an Excel file and a CSV

In [2]:
#Reading plot centers from CSV
lfi_plot_path = r".\Data\Original_Data\LFI\anzahlBaeumeLFI_from_p\plot_KoordinatenWaldrBestgren_20201019.csv"

lfi_plots = pd.read_csv(lfi_plot_path, delimiter=";")

In [3]:
#Derive geodataframe from CSV data
geometry = [Point(xy) for xy in zip(lfi_plots.X, lfi_plots.Y)]
lfi_plots = gpd.GeoDataFrame(lfi_plots, crs= {'init': "epsg:21781"}, geometry=geometry)
#Convert to LV95 and derive common attributes
lfi_plots = lfi_plots.to_crs(epsg=2056)
lfi_plots["x_lv95"] = lfi_plots.geometry.x
lfi_plots["y_lv95"] = lfi_plots.geometry.y
lfi_plots["plot_area"] = math.pi*12.62**2

C:\ProgramData\Miniconda3\envs\lab\lib\site-packages\pyproj\crs\crs.py:131: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)


In [4]:
lfi_plots.head()

,INVNR,CLNR,X,Y,WALDRAND,BESTANDESGRENZE,ANZ_STEHEND_LEBEND_BAUM_HA,geometry,x_lv95,y_lv95,plot_area
0,450,6,685998.398,295009.246,1,2,115.444455,POINT (2685999.604 1295008.949),2.686000e+06,1.295009e+06,500.343869
1,450,22,685021.881,293988.559,0,1,340.000000,POINT (2685023.082 1293988.265),2.685023e+06,1.293988e+06,500.343869
2,450,26,686992.655,293997.707,0,2,390.000000,POINT (2686993.855 1293997.405),2.686994e+06,1.293997e+06,500.343869
3,450,31,690996.792,293999.263,1,2,470.000000,POINT (2690997.989 1293998.940),2.690998e+06,1.293999e+06,500.343869
4,450,47,686002.003,293001.178,0,2,610.000000,POINT (2686003.199 1293000.879),2.686003e+06,1.293001e+06,500.343869


In [5]:
#Reading individual tree data from Excel
reference_trees_path = r".\Data\Original_Data\LFI\ReferenzBaeume\Referenzdaten_Baeume_20200312x_mod.xlsx"

lfi_trees = pd.read_excel(reference_trees_path,sheet_name="Baumtabelle_Referenz", engine="openpyxl")

In [6]:
#Filtering out only the trees belonging to LFI
lfi_trees = lfi_trees[lfi_trees["Quelle"]==4]

#Derive basal area per tree
lfi_trees["bas"] = lfi_trees["DBHcm"].apply(lambda x: (x/200)**2*math.pi)

In [7]:
#Determine total number of trees and basal area per plot
lfi_trees["nbr_trees"] = lfi_trees.groupby("ID_Flaeche")["ID_Flaeche"].transform('count')
lfi_trees["bas_sum"] = lfi_trees.groupby("ID_Flaeche")["bas"].transform('sum')

In [8]:
#Aggregate values per plot
lfi_trees_grouped = lfi_trees.groupby("ID_Flaeche").min().reset_index()
#lfi_trees_grouped = lfi_trees_grouped[lfi_trees_grouped["bas_sum"]!=0]

In [9]:
print(len(lfi_plots))
print(len(lfi_plots[np.invert(np.isnan(lfi_plots["CLNR"]))]))
print(len(lfi_plots[(np.isnan(lfi_plots["CLNR"]))]))


6034
6034
0


In [10]:
#Merge plot centers and grouped tree information from excel
print(len(lfi_plots))
lfi_plots = lfi_plots.merge(lfi_trees_grouped[["ID_Flaeche","nbr_trees","bas_sum","Aufnahmejahr"]], left_on=["CLNR"], right_on=["ID_Flaeche"], how="left")
print(len(lfi_plots))

print(len(lfi_plots[np.invert(np.isnan(lfi_plots["CLNR"]))]))

6034
6034
6034


In [11]:
#Read individual tree information with layer attribute from CSV 
lfi_tree_layer_path = r".\Data\Original_Data\LFI\anzahlBaeumeLFI_from_p\baum_Schicht_20201019.csv"
lfi_tree_layer = pd.read_csv(lfi_tree_layer_path, delimiter=";")
lfi_tree_layer["ID_Baum"]=lfi_tree_layer["BANR"]

In [12]:
#Derive separate number of trees per plot 

#Upper layer
lfi_tree_layer["IsOberschichtLFI"]=lfi_tree_layer["SCHICHT"].apply(lambda x: 1 if x==1 else 0)
lfi_tree_layer["stems_in_plot_osLFI"] = lfi_tree_layer.groupby(["CLNR"])["IsOberschichtLFI"].transform("sum")

#Middle Layer
lfi_tree_layer["IsMittelschichtLFI"]=lfi_tree_layer["SCHICHT"].apply(lambda x: 1 if x==2 else 0)
lfi_tree_layer["stems_in_plot_msLFI"] = lfi_tree_layer.groupby(["CLNR"])["IsMittelschichtLFI"].transform("sum")

#Lower Layer
lfi_tree_layer["IsUnterschichtLFI"]=lfi_tree_layer["SCHICHT"].apply(lambda x: 1 if x==3 else 0)
lfi_tree_layer["stems_in_plot_usLFI"] = lfi_tree_layer.groupby(["CLNR"])["IsUnterschichtLFI"].transform("sum")

#No Layer
lfi_tree_layer["IsKeineSchichtLFI"]=lfi_tree_layer["SCHICHT"].apply(lambda x: 1 if x==4 else 0)
lfi_tree_layer["stems_in_plot_ksLFI"] = lfi_tree_layer.groupby(["CLNR"])["IsKeineSchichtLFI"].transform("sum")

#No value for Layer attribute
lfi_tree_layer["IsKwSchichtLFI"]=lfi_tree_layer["SCHICHT"].apply(lambda x: 1 if x==-1 else 0)
lfi_tree_layer["stems_in_plot_kwsLFI"] = lfi_tree_layer.groupby(["CLNR"])["IsKwSchichtLFI"].transform("sum")

#Derive total number of trees per plot
lfi_tree_layer["stems_in_plot_LFI"] = lfi_tree_layer.groupby(["CLNR"])["CLNR"].transform("count")

#Aggregate values per plot
lfi_tree_layer_grouped = lfi_tree_layer.groupby("CLNR").aggregate("min").reset_index()

In [13]:
print(lfi_tree_layer["stems_in_plot_LFI"].describe())
print(lfi_tree_layer["stems_in_plot_osLFI"].describe())

count    82346.000000
mean        17.694703
std          7.707497
min          1.000000
25%         12.000000
50%         17.000000
75%         22.000000
max         56.000000
Name: stems_in_plot_LFI, dtype: float64
count    82346.000000
mean        10.525417
std          6.172101
min          0.000000
25%          6.000000
50%         10.000000
75%         14.000000
max         50.000000
Name: stems_in_plot_osLFI, dtype: float64


In [14]:
#Merge layer information with plot centers
lfi_plots = lfi_plots.merge(lfi_tree_layer_grouped[['CLNR', 'stems_in_plot_osLFI', 'stems_in_plot_usLFI', 'stems_in_plot_ksLFI', 'stems_in_plot_kwsLFI', 'stems_in_plot_LFI', 'stems_in_plot_msLFI']], suffixes = ("","_lfiSchicht"), left_on="CLNR", right_on="CLNR",how="left")


In [15]:
#Derive common reference attributes 
lfi_plots["density_actual"] =  lfi_plots["stems_in_plot_osLFI"]/(lfi_plots["plot_area"]/10000) #!!!!
lfi_plots["nbr_trees_os"] = lfi_plots["stems_in_plot_osLFI"]
lfi_plots["density_actual_all"] =  lfi_plots["ANZ_STEHEND_LEBEND_BAUM_HA"]
lfi_plots["bas_per_ha"] =  lfi_plots["bas_sum"]/(lfi_plots["plot_area"]/10000)

lfi_plots["density_actual"]= lfi_plots.apply(lambda x: x["density_actual_all"] if np.isnan(x["density_actual"]) else x["density_actual"],axis=1)

In [16]:
#Export plot center data as basis for detection and analysis 
csv_out_path = r".\Data\reference_plots_lfi_raw.csv"
lfi_plots.to_csv(csv_out_path, sep=";", header=True, quotechar='"', index=False)

In [17]:
lfi_trees.columns

Index(['ID_Baum', 'ID_Flaeche', 'X_LV03', 'Y_LV03', 'X_LV95', 'Y_LV95', 'Z',
       'Baumart_LFI', 'DBHcm', 'Hoehe_Gemessen', 'Kluppschwelle',
       'SozialeStellung', 'Kronenbreite', 'Kronenlaengenanteil', 'Lebend',
       'Quelle', 'MethodeVermessungFlaeche', 'GenauigkeitFlaeche',
       'MethodeVermessungBaum', 'GenauigkeitBaum', 'Aufnahmejahr',
       'Aufnahmedatum', 'Kontakt', 'bas', 'nbr_trees', 'bas_sum'],
      dtype='object')

In [18]:
reference_trees_layer_columnsh = r".\Data\Original_Data\LFI\AP07__Kalibrieren_der_EBD_Regeln\LFI_Baeume_Schicht.xlsx"
reference_trees_layer = pd.read_excel(reference_trees_layer_columnsh,sheet_name="LFI_Schicht", engine="openpyxl")

In [19]:
geometry = [Point(xy) for xy in zip(reference_trees_layer.GPS_X, reference_trees_layer.GPS_Y)]
reference_trees_layer = gpd.GeoDataFrame(reference_trees_layer, crs= {'init': "epsg:21781"}, geometry=geometry)

C:\ProgramData\Miniconda3\envs\lab\lib\site-packages\pyproj\crs\crs.py:131: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)


In [20]:
reference_trees_layer = reference_trees_layer.to_crs(epsg=2056)

In [21]:
shp_out_path = r".\Data\reference_trees_layer.shp"
reference_trees_layer.to_file(shp_out_path)

## Combine data from different datasources

In [22]:
#Select common attributes per data source, rename columns to common scheme, assign ID per source
lfi_plots_selection = lfi_plots[["CLNR","x_lv95","y_lv95","WALDRAND","BESTANDESGRENZE","nbr_trees","nbr_trees_os","bas_sum","Aufnahmejahr","density_actual","density_actual_all","bas_per_ha"]].reset_index(drop=True)
lfi_plots_selection.rename(columns={"CLNR":"plot_id", "bas_sum":"bas", "Aufnahmejahr":"year","WALDRAND":"waldrand","BESTANDESGRENZE":"bestandesgrenze"}, inplace=True)
lfi_plots_selection["source_id"] = 1
print(len(lfi_plots_selection))

#Merge data from data sources into one dataframe
merged_plots = lfi_plots_selection

#Derive geodataframe
geometry = [Point(xy) for xy in zip(merged_plots.x_lv95, merged_plots.y_lv95)]
merged_plots = gpd.GeoDataFrame(merged_plots, crs= {'init': "epsg:2056"}, geometry=geometry)

6034


In [23]:
#Read borders for cantons and municipalities 
kantone_path = r"P:\HAFL\9 Share\PermanenteOrdner\Geodaten\Nationale_Daten\Nationale_Daten_SWISSTOPO\SwissBounderies_2013\BOUNDARIES_2013\swissBOUNDARIES3D\SHAPEFILE_LV95_LN02\swissBOUNDARIES3D_1_1_TLM_KANTONSGEBIET.shp"
gemeinden_path = r"P:\HAFL\9 Share\PermanenteOrdner\Geodaten\Nationale_Daten\Nationale_Daten_SWISSTOPO\SwissBounderies_2013\BOUNDARIES_2013\swissBOUNDARIES3D\SHAPEFILE_LV95_LN02\swissBOUNDARIES3D_1_1_TLM_HOHEITSGEBIET.shp"

kantone = gpd.read_file(kantone_path)
gemeinden = gpd.read_file(gemeinden_path)


In [24]:
merged_plots

,plot_id,x_lv95,y_lv95,waldrand,bestandesgrenze,nbr_trees,nbr_trees_os,bas,year,density_actual,density_actual_all,bas_per_ha,source_id,geometry
0,6,2.686000e+06,1.295009e+06,1,2,2.0,2.0,0.325233,2015.0,39.972509,115.444455,6.500197,1,POINT (2685999.604 1295008.949)
1,22,2.685023e+06,1.293988e+06,0,1,12.0,11.0,1.236374,2016.0,219.848802,340.000000,24.710481,1,POINT (2685023.082 1293988.265)
2,26,2.686994e+06,1.293997e+06,0,2,12.0,11.0,1.031935,2010.0,219.848802,390.000000,20.624509,1,POINT (2686993.855 1293997.405)
3,31,2.690998e+06,1.293999e+06,1,2,13.0,6.0,1.200389,2016.0,119.917528,470.000000,23.991280,1,POINT (2690997.989 1293998.940)
4,47,2.686003e+06,1.293001e+06,0,2,17.0,14.0,1.468966,2013.0,279.807566,610.000000,29.359124,1,POINT (2686003.199 1293000.879)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6029,164999,2.722027e+06,1.080982e+06,0,2,5.0,5.0,0.109327,2012.0,99.931273,200.000000,2.185046,1,POINT (2722027.054 1080982.070)
6030,165003,2.723995e+06,1.081007e+06,0,2,11.0,10.0,0.987629,2016.0,199.862547,340.000000,19.738998,1,POINT (2723995.432 1081006.736)
6031,165118,2.723999e+06,1.079004e+06,1,1,5.0,3.0,0.628319,2017.0,59.958764,282.649791,12.557734,1,POINT (2723998.973 1079004.073)
6032,165139,2.715001e+06,1.077998e+06,0,2,4.0,1.0,0.122758,2016.0,19.986255,150.000000,2.453467,1,POINT (2715001.103 1077998.348)


In [25]:
#Merge plot centers with cantons to determine the surrounding canton (important for picking inputs during detection)
merged_plots = sjoin(merged_plots, gemeinden[["KANTONSNUM","BFS_NUMMER","geometry"]], how='left')
merged_plots = merged_plots.drop(columns=["index_right"])

In [26]:
# Add running ID
merged_plots = merged_plots.reset_index()
merged_plots = merged_plots.rename(columns={"index":"OBJECTID"})

In [27]:
#Read reference data about cantons and detection inputs
kantone_info_path = r".\Data\Original_Data\kantone_info_vhm_schweizweit.csv"

kantone_info = pd.read_csv(kantone_info_path, delimiter=";")

In [28]:
#Merge detection reference data about ALS acquisition year with plot centers
merged_plots = merged_plots.merge(kantone_info[["Flaeche_ID","VHM_Year_Min","VHM_Year_Max"]], left_on=["KANTONSNUM"], right_on=["Flaeche_ID"], how="left")


In [29]:
merged_plots.columns

Index(['OBJECTID', 'plot_id', 'x_lv95', 'y_lv95', 'waldrand',
       'bestandesgrenze', 'nbr_trees', 'nbr_trees_os', 'bas', 'year',
       'density_actual', 'density_actual_all', 'bas_per_ha', 'source_id',
       'geometry', 'KANTONSNUM', 'BFS_NUMMER', 'Flaeche_ID', 'VHM_Year_Min',
       'VHM_Year_Max'],
      dtype='object')

In [30]:
#Drop unneeded columns
merged_plots = merged_plots.drop(columns=["BFS_NUMMER","Flaeche_ID"])

In [31]:
#Read metadata for ALS tiles 
als_meta_path = r".\Data\reference_plot_als_meta.csv"

als_meta = pd.read_csv(als_meta_path, delimiter=";")



In [32]:
#Merge plot centers with cantons to determine the surrounding canton (important for picking inputs during detection)
merged_plots = merged_plots.merge(als_meta[['plot_id', 'source_id', 'file_name']], suffixes = ("","_als"), left_on=["plot_id","source_id"], right_on=["plot_id","source_id"],how="left")


In [33]:
merged_plots["file_name"].apply(lambda s: s[-21:-17]).unique()

array(['2018', '2017', '2020', '2006', '2019', '2001', 'O_xx', '2011',
       '2007', '2012', '2014', '2003', '2002', '2013', '2005'],
      dtype=object)

In [34]:
merged_plots["VHM_Tile_Year"] = merged_plots.apply(lambda s: int(s["file_name"][-21:-17]) if not ("xx" in s["file_name"][-21:-17]) else s["VHM_Year_Min"] ,axis=1)
merged_plots = merged_plots.drop(columns=["file_name"])


In [35]:
merged_plots.head()

,OBJECTID,plot_id,x_lv95,y_lv95,waldrand,bestandesgrenze,nbr_trees,nbr_trees_os,bas,year,density_actual,density_actual_all,bas_per_ha,source_id,geometry,KANTONSNUM,VHM_Year_Min,VHM_Year_Max,VHM_Tile_Year
0,0,6,2.686000e+06,1.295009e+06,1,2,2.0,2.0,0.325233,2015.0,39.972509,115.444455,6.500197,1,POINT (2685999.604 1295008.949),14.0,2018,2018,2018
1,1,22,2.685023e+06,1.293988e+06,0,1,12.0,11.0,1.236374,2016.0,219.848802,340.000000,24.710481,1,POINT (2685023.082 1293988.265),14.0,2018,2018,2018
2,2,26,2.686994e+06,1.293997e+06,0,2,12.0,11.0,1.031935,2010.0,219.848802,390.000000,20.624509,1,POINT (2686993.855 1293997.405),14.0,2018,2018,2018
3,3,31,2.690998e+06,1.293999e+06,1,2,13.0,6.0,1.200389,2016.0,119.917528,470.000000,23.991280,1,POINT (2690997.989 1293998.940),14.0,2018,2018,2018
4,4,47,2.686003e+06,1.293001e+06,0,2,17.0,14.0,1.468966,2013.0,279.807566,610.000000,29.359124,1,POINT (2686003.199 1293000.879),14.0,2018,2018,2018


In [36]:
#Export plot center data as basis for detection and analysis 
csv_out_path = r".\Data\reference_plots.csv"
merged_plots.to_csv(csv_out_path, sep=";", header=True, quotechar='"', index=False)
shp_out_path = r".\Data\reference_plots.shp"
merged_plots.to_file(shp_out_path)

C:\Users\admin\AppData\Local\Temp/ipykernel_22252/2867745002.py:5: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  merged_plots.to_file(shp_out_path)
